Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

C:\Program Files\Microsoft SQL Server\MSSQL14.MSSQLSERVER\PYTHON_SERVICES\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First reload the data we generated in `1_notmnist.ipynb`.

In [7]:
import os
data_root = r"D:\study\DeepLearning\NotMnist"
pickle_file = os.path.join(data_root,'notMNIST.pickle')


with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  train_labels_log=train_labels
  valid_labels_log=valid_labels
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [12]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [24]:
#introducing regularization to Logistic CLassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
subset_size=5000
train_data_subset=train_dataset[0:subset_size,:]
train_labels_subset=train_labels_log[0:subset_size]
model=LogisticRegression(multi_class='multinomial',solver='lbfgs',penalty='l2',C=.01)
model.fit(train_data_subset,train_labels_subset)
predict_labels=model.predict(valid_dataset)
score=accuracy_score(predict_labels,valid_labels_log)
print(score)

0.8163


In [27]:
#introducing regularization to Neural Network Model
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
batch_size = 128
logpath='./tflogs/'
Hidden_layer_Nodes=1024
Beta=.01
graph=tf.Graph()
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)
with graph.as_default():
      tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size),name="Training_Features")
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels),name="Training_Labels")
      print(tf_train_dataset)
      print(tf_train_labels)  
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)
      # Variables.
      #Layer1
      with tf.name_scope("Layer1"):
          with tf.name_scope("Weights"):
              weights_L1 = tf.Variable(tf.truncated_normal([image_size * image_size, Hidden_layer_Nodes]))
              variable_summaries(weights_L1)
          with tf.name_scope("Biases"):
              biases_L1 = tf.Variable(tf.zeros([Hidden_layer_Nodes]))
              variable_summaries(biases_L1)
      #Layer2
      with tf.name_scope("Layer2"):
          with tf.name_scope("Weights"):
              weights_L2 = tf.Variable(tf.truncated_normal([Hidden_layer_Nodes, num_labels]))
              variable_summaries(weights_L2)
          with tf.name_scope("Biases"):
              biases_L2 = tf.Variable(tf.zeros([num_labels]))
              variable_summaries(weights_L2)
      # Training computation.      
      logits_L1 = tf.matmul(tf_train_dataset, weights_L1) + biases_L1
      tf.summary.histogram('Layer1_Logits',logits_L1)
      Relu=tf.nn.relu(logits_L1)
      tf.summary.histogram('Activations_Relu',Relu)
      logits_L2 = tf.matmul(Relu, weights_L2) + biases_L2
      tf.summary.histogram('Layer2_Logits',logits_L2)
      loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_L2))
      regularizers=tf.nn.l2_loss(weights_L1)+ tf.nn.l2_loss(weights_L2)
      loss = loss + Beta*regularizers
      tf.summary.histogram('Loss',loss)
      # Optimizer.
      with tf.name_scope("Train"):
          optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits_L2)
      valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_L1) + biases_L1),weights_L2)+biases_L2)
      test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_L1) + biases_L1),weights_L2)+biases_L2)
      #Merge Summaries
      summarize_all=tf.summary.merge_all()




Tensor("Training_Features:0", shape=(128, 784), dtype=float32)
Tensor("Training_Labels:0", shape=(128, 10), dtype=float32)


In [29]:

num_steps = 1001
with tf.Session(graph=graph) as session:
  tfWriter=tf.summary.FileWriter(logpath,session.graph)
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions,summary = session.run([optimizer, loss, train_prediction,summarize_all], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
      tfWriter.add_summary(summary,step)



Initialized
Minibatch loss at step 0: 3453.850342
Minibatch accuracy: 12.5%
Validation accuracy: 30.6%
Test accuracy: 33.7%
Minibatch loss at step 100: 1152.986328
Minibatch accuracy: 75.0%
Validation accuracy: 78.8%
Test accuracy: 86.5%
Minibatch loss at step 200: 424.650299
Minibatch accuracy: 77.3%
Validation accuracy: 76.2%
Test accuracy: 83.2%
Minibatch loss at step 300: 153.834778
Minibatch accuracy: 78.1%
Validation accuracy: 82.0%
Test accuracy: 88.9%
Minibatch loss at step 400: 56.721504
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Test accuracy: 90.6%
Minibatch loss at step 500: 21.199116
Minibatch accuracy: 87.5%
Validation accuracy: 83.7%
Test accuracy: 90.7%
Minibatch loss at step 600: 8.350582
Minibatch accuracy: 84.4%
Validation accuracy: 83.7%
Test accuracy: 90.7%
Minibatch loss at step 700: 3.631584
Minibatch accuracy: 80.5%
Validation accuracy: 83.1%
Test accuracy: 90.4%
Minibatch loss at step 800: 1.658815
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [31]:
num_steps = 1001
train_data_subset=train_dataset[0:5000,:]
train_labels_subset=train_labels[0:5000]
with tf.Session(graph=graph) as session:
  tfWriter=tf.summary.FileWriter(logpath,session.graph)
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_subset.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_data_subset[offset:(offset + batch_size), :]
    batch_labels = train_labels_subset[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions,summary = session.run([optimizer, loss, train_prediction,summarize_all], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
      tfWriter.add_summary(summary,step)

Initialized
Minibatch loss at step 0: 3491.104492
Minibatch accuracy: 11.7%
Validation accuracy: 33.3%
Test accuracy: 35.8%
Minibatch loss at step 100: 1150.539551
Minibatch accuracy: 91.4%
Validation accuracy: 79.4%
Test accuracy: 86.7%
Minibatch loss at step 200: 421.762451
Minibatch accuracy: 96.1%
Validation accuracy: 78.7%
Test accuracy: 85.8%
Minibatch loss at step 300: 154.681000
Minibatch accuracy: 100.0%
Validation accuracy: 80.9%
Test accuracy: 88.0%
Minibatch loss at step 400: 56.931324
Minibatch accuracy: 100.0%
Validation accuracy: 81.8%
Test accuracy: 88.8%
Minibatch loss at step 500: 21.277807
Minibatch accuracy: 98.4%
Validation accuracy: 83.3%
Test accuracy: 90.4%
Minibatch loss at step 600: 8.252136
Minibatch accuracy: 96.9%
Validation accuracy: 83.3%
Test accuracy: 90.3%
Minibatch loss at step 700: 3.496755
Minibatch accuracy: 93.8%
Validation accuracy: 82.1%
Test accuracy: 89.4%
Minibatch loss at step 800: 1.862516
Minibatch accuracy: 88.3%
Validation accuracy: 83.3

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
